In [1]:
def draw_label(input_image, label, left, top):
    """Draw text onto image at location."""
    
    # Get text size.
    text_size = cv2.getTextSize(label, FONT_FACE, FONT_SCALE, THICKNESS)
    dim, baseline = text_size[0], text_size[1]
    cv2.putText(input_image, label, (left, top + dim[1]), FONT_FACE, FONT_SCALE, BLACK, THICKNESS, cv2.LINE_AA)


def pre_process(input_image, net):
    # Create a 4D blob from a frame.
    arrayOutput = []
    for cams in input_image:
        blob = cv2.dnn.blobFromImage(cams, 1/255.0, (INPUT_WIDTH, INPUT_HEIGHT), [0,0,0], 1, crop=False)
        
        # Sets the input to the network.
        net.setInput(blob)

        # Runs the forward pass to get output of the output layers.
        output_layers = net.getUnconnectedOutLayersNames()
        outputs = net.forward(output_layers)
        
        arrayOutput.append(outputs)
        # print(outputs[0].shape)
        
    return (arrayOutput)

def post_process(input_image, outputs, CamUso):
    # Lists to hold respective values while unwrapping.
    global row, image_height
    class_ids = []
    confidences = []
    boxes = []
    cont = 0
    for cams in outputs:
        # Rows.
        rows = cams[0].shape[1]
        image_height, image_width = input_image[cont].shape[:2]

        # Resizing factor.
        x_factor = image_width / INPUT_WIDTH
        y_factor =  image_height / INPUT_HEIGHT
        
        # Iterate through 25200 detections.
        for r in range(rows):
            row = cams[0][0][r]
            confidence = row[4]
            
            #confidence = max(confidence)
            # Discard bad detections and continue.
            if confidence >= CONFIDENCE_THRESHOLD:
                classes_scores = row[5:]
                
                # Get the index of max class score.
                class_id = np.argmax(classes_scores)

                #  Continue if the class score is above threshold.
                if (classes_scores[class_id] > SCORE_THRESHOLD):
                    confidences.append(confidence)
                    class_ids.append(class_id)

                    cx, cy, w, h = row[0], row[1], row[2], row[3]

                    left = int((cx - w/2) * x_factor)
                    top = int((cy - h/2) * y_factor)
                    width = int(w * x_factor)
                    height = int(h * y_factor)

                    box = np.array([left, top, width, height])
                    boxes.append(box)

        # Perform non maximum suppression to eliminate redundant overlapping boxes with
        # lower confidences.
        cont += 1

    Linha = 10
    ArrayProd = []
    
    indices = cv2.dnn.NMSBoxes(boxes, confidences, CONFIDENCE_THRESHOLD, NMS_THRESHOLD)
    for i in indices:
        if classes[class_ids[i]] not in ArrayProd: 
            box = boxes[i]
            left = box[0]
            top = box[1]
            width = box[2]
            height = box[3]

            label = "{}:".format(classes[class_ids[i]])
            ArrayProd.append(classes[class_ids[i]])
            
            #cv2.circle(CamUso, (int(left + width / 2) , int(top + height / 2)), 12, (0, 0, 255), -1)
            draw_label(CamUso, label, 20, Linha)
            Linha += 25
            
    return(CamUso)


def build_model(is_cuda):
#    os.environ['CUDA_VISIBLE_DEVICES'] = '0, 1'
    
    modelWeights = "/home/pantha/Autonomoous/Pesos/last5s_640_4Prod_ROI.onnx"
    net = cv2.dnn.readNet(modelWeights)

    if is_cuda:
        net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
        net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)
        print("[INFO] setting preferable backend and target to CUDA...")
    else:
        net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
        net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)
        print("[INFO] setting preferable backend and target to CPU...")
        
    return (net)

In [2]:
from __future__ import print_function
from imutils.video import WebcamVideoStream
from imutils.video import FPS
import argparse
import imutils
import cv2
import torch
import numpy as np

# Constants.
INPUT_WIDTH = 640
INPUT_HEIGHT = 640
SCORE_THRESHOLD = 0.60
NMS_THRESHOLD = 0.55
CONFIDENCE_THRESHOLD = 0.55

# Text parameters.
FONT_FACE = cv2.FONT_HERSHEY_SIMPLEX
FONT_SCALE = 0.75
THICKNESS = 1

# Colors
BLACK  = (0,0,0)
BLUE   = (255,178,50)
YELLOW = (0,255,255)
RED = (0,0,255)
BRANCO = (255,255,255)

colors = [(255, 255, 0), (0, 255, 0), (0, 255, 255), (255, 0, 0)]

is_cuda = 'cuda' if torch.cuda.is_available() else 'cpu'

net = build_model(is_cuda)

# Load class names.
classesFile = "/home/pantha/Autonomoous/Dataset/Prod_Pantha/classes.txt"
classes = None

with open(classesFile, 'rt') as f:
    classes = f.read().rstrip('\n').split('\n')



#stream = cv2.VideoCapture("rtsp://192.168.0.18:554/user=admin_password=tlJwpbo6_channel=1_stream=0.sdp?real_stream")
EndCam11 = "rtsp://192.168.0.11:554/user=admin_password=M1IUut80_channel=1_stream=0.sdp?real_stream"
EndCam15 = "rtsp://192.168.0.15:554/user=admin_password=tlJwpbo6_channel=1_stream=0.sdp?real_stream"
EndCam17 = "rtsp://192.168.0.17:554/user=admin_password=tlJwpbo6_channel=1_stream=0.sdp?real_stream"
EndCam18 = "rtsp://192.168.0.18:554/user=admin_password=tlJwpbo6_channel=1_stream=0.sdp?real_stream"
EndCam19 = "rtsp://192.168.0.19:554/user=admin_password=tlJwpbo6_channel=1_stream=0.sdp?real_stream"
#EndCam19 = "/home/pantha/Autonomoous/Dataset/Video/Sucrilhos_Nescau_1.mp4"

print("[INFO] sampling THREADED frames from webcam...")
vs11 = WebcamVideoStream(src=EndCam11).start()
vs15 = WebcamVideoStream(src=EndCam15).start()
vs17 = WebcamVideoStream(src=EndCam17).start()
vs18 = WebcamVideoStream(src=EndCam18).start()
vs19 = WebcamVideoStream(src=EndCam19).start()

fps = FPS().start()
# loop over some frames...this time using the threaded stream
while True:
    # grab the frame from the threaded video stream and resize it
    # to have a maximum width of 400 pixel
    arrayFrameCams = []
    arrayReadCams = []
    
    frame11 = vs11.read()
    frame15 = vs15.read()
    frame17 = vs17.read()
    frame18 = vs18.read()
    frame19 = vs19.read()
    
    ref_frame11 = imutils.resize(frame11, width=400)
    ref_frame15 = imutils.resize(frame15, width=400)
    ref_frame17 = imutils.resize(frame17, width=400)
    ref_frame18 = imutils.resize(frame18, width=400)
    ref_frame19 = imutils.resize(frame19, width=400)
    
    ref_frame11 = cv2.cvtColor(ref_frame11, cv2.COLOR_BGRA2BGR)
    ref_frame15 = cv2.cvtColor(ref_frame15, cv2.COLOR_BGRA2BGR)
    ref_frame17 = cv2.cvtColor(ref_frame17, cv2.COLOR_BGRA2BGR)
    ref_frame18 = cv2.cvtColor(ref_frame18, cv2.COLOR_BGRA2BGR)
    ref_frame19 = cv2.cvtColor(ref_frame19, cv2.COLOR_BGRA2BGR)

    arrayFrameCams.append(ref_frame11)
    arrayFrameCams.append(ref_frame15)
    arrayFrameCams.append(ref_frame17)
    arrayFrameCams.append(ref_frame18)
    arrayFrameCams.append(ref_frame19)
    
    arrayReadCams.append(frame11)
    arrayReadCams.append(frame15)
    arrayReadCams.append(frame17)
    arrayReadCams.append(frame18)
    arrayReadCams.append(frame19)
      
    detections = pre_process(arrayFrameCams, net)
    img = post_process(arrayReadCams, detections, frame17)
    
    output = cv2.resize(img, (800, 600))
    
    cv2.imshow("PanthaVentures - Autonomous", output)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    # update the FPS counter
    fps.update()

# stop the timer and display FPS information
fps.stop()

print("[INFO] elasped time: {:.2f}".format(fps.elapsed()))
print("[INFO] approx. FPS: {:.2f}".format(fps.fps()))
# do a bit of cleanup
print("[INFO] Stop Cameras")

vs11.stop()
vs15.stop()
vs17.stop()
vs18.stop()
vs19.stop()
cv2.destroyAllWindows()


[INFO] setting preferable backend and target to CUDA...
[INFO] sampling THREADED frames from webcam...
[INFO] elasped time: 37.39
[INFO] approx. FPS: 2.92
[INFO] Stop Cameras
